In [1]:
import os

In [3]:
os.chdir('../')


In [5]:
from dataclasses import dataclass
from pathlib import Path

from swahiliNewsClassifier.constants import *
from swahiliNewsClassifier.utilities.helper_functions import read_yaml, create_directories

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    train_source_URL: str
    test_source_URL: str
    train_data_file: Path
    test_data_file: Path
    decompressed_dir: Path

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        """
        Initialize ConfigurationManager with configuration and parameter files.

        Args:
            config_filepath (str): Path to the configuration YAML file.
            params_filepath (str): Path to the parameters YAML file.
        """
        self.configurations = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Get the data ingestion configuration.

        Returns:
            DataIngestionConfig: Configuration object for data ingestion.
        """
        data_ingestion_configurations = self.configurations.data_ingestion

        create_directories([data_ingestion_configurations.root_dir])

        return DataIngestionConfig(
            root_dir=data_ingestion_configurations.root_dir,
            train_source_URL=data_ingestion_configurations.train_source_URL,
            test_source_URL=data_ingestion_configurations.test_source_URL,
            train_data_file=data_ingestion_configurations.train_data_file,
            test_data_file=data_ingestion_configurations.test_data_file,
            decompressed_dir=data_ingestion_configurations.decompressed_dir
        )


In [8]:
import os
import zipfile
import gdown
from swahiliNewsClassifier import log

In [9]:
class DataIngestion:
    def __init__(self, data_ingestion_configuartions: DataIngestionConfig):
        """
        Initialize DataIngestion object with the provided configuration.

        Args:
            data_ingestion_configuartions (DataIngestionConfig): Configuration object for data ingestion.
        """
        self.data_ingestion_configuartions = data_ingestion_configuartions

    def download_file(self):
        """Fetch data from a URL.
        
        Raises:
            Exception: If an error occurs during the download process.
        """
        os.makedirs("artifacts/data_ingestion/compressed", exist_ok=True)
        os.makedirs("artifacts/data_ingestion/decompressed", exist_ok=True)
        dataset_urls = [self.data_ingestion_configuartions.train_source_URL, self.data_ingestion_configuartions.test_source_URL]
        zip_download_dir = [self.data_ingestion_configuartions.train_data_file, self.data_ingestion_configuartions.test_data_file]
        
        for url, dest in zip(dataset_urls, zip_download_dir):
            try:
                log.info(f"Downloading data from {url} into file {dest}")

                file_id = url.split("/")[-2]
                prefix = "https://drive.google.com/uc?/export=download&id="
                gdown.download(prefix + file_id, dest)

                log.info(f"Downloaded data from {url} into file {dest}")
            except Exception as e:
                log.error(f"Error downloading file from {url} to {dest}")
                raise e

    def extract_zip_file(self):
        """Extract a zip file.

        This method extracts the contents of a zip file specified in the configuration
        to the directory specified in the configuration.

        Raises:
            Exception: If an error occurs during the extraction process.
        """
        zip_download_dir = [self.data_ingestion_configuartions.train_data_file, self.data_ingestion_configuartions.test_data_file]
        decompress_path = self.data_ingestion_configuartions.decompressed_dir
        os.makedirs(decompress_path, exist_ok=True)

        for zip_file in zip_download_dir:
            try:
                with zipfile.ZipFile(zip_file, "r") as zip_ref:
                    zip_ref.extractall(decompress_path)

                log.info(f"Extracted zip file {zip_file} into: {decompress_path}")
            except Exception as e:
                log.error(f"Error extracting zip file: {zip_file}")
                raise e


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-06-08 17:23:44,925: INFO: helper_functions:20: yaml file: configuration/configuration.yaml loaded successfully]
[2024-06-08 17:23:44,935: INFO: helper_functions:20: yaml file: parameters.yaml loaded successfully]
[2024-06-08 17:23:44,942: INFO: helper_functions:35: Created directory at: artifacts]
[2024-06-08 17:23:44,945: INFO: helper_functions:35: Created directory at: artifacts/data_ingestion]
[2024-06-08 17:23:44,952: INFO: 3987317555:24: Downloading data from https://drive.google.com/file/d/15stuLDZkXNOgBUC1rnx5yXYdVPViUjNB/view?usp=sharing into file artifacts/data_ingestion/compressed/train_data.zip]


Downloading...
From: https://drive.google.com/uc?/export=download&id=15stuLDZkXNOgBUC1rnx5yXYdVPViUjNB
To: /media/kalema/9954-79C8/Projects/Swahili-News-Classifier/artifacts/data_ingestion/compressed/train_data.zip
100%|██████████| 3.78M/3.78M [00:03<00:00, 946kB/s] 

[2024-06-08 17:23:52,445: INFO: 3987317555:30: Downloaded data from https://drive.google.com/file/d/15stuLDZkXNOgBUC1rnx5yXYdVPViUjNB/view?usp=sharing into file artifacts/data_ingestion/compressed/train_data.zip]
[2024-06-08 17:23:52,446: INFO: 3987317555:24: Downloading data from https://drive.google.com/file/d/1mjmYzMdnn_UwSEgTQ7i-cJ5WSOokt9Er/view?usp=sharing into file artifacts/data_ingestion/compressed/test_data.zip]



Downloading...
From: https://drive.google.com/uc?/export=download&id=1mjmYzMdnn_UwSEgTQ7i-cJ5WSOokt9Er
To: /media/kalema/9954-79C8/Projects/Swahili-News-Classifier/artifacts/data_ingestion/compressed/test_data.zip
100%|██████████| 992k/992k [00:00<00:00, 1.15MB/s]

[2024-06-08 17:23:56,279: INFO: 3987317555:30: Downloaded data from https://drive.google.com/file/d/1mjmYzMdnn_UwSEgTQ7i-cJ5WSOokt9Er/view?usp=sharing into file artifacts/data_ingestion/compressed/test_data.zip]
[2024-06-08 17:23:56,455: INFO: 3987317555:53: Extracted zip file artifacts/data_ingestion/compressed/train_data.zip into: artifacts/data_ingestion/decompressed]


[2024-06-08 17:23:56,535: INFO: 3987317555:53: Extracted zip file artifacts/data_ingestion/compressed/test_data.zip into: artifacts/data_ingestion/decompressed]
